### Topic modelling

See *Blei, 2003: Latent dirichlet allocation* [PDF](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) for a description of LDA.

### LDA Topic modelling



#### What is The Jupyter Project and Jupyter Notebook

> Project [Jupyter](http://jupyter.org/) develops open-source software for **interactive and reproducible computing**.<br>
> The **open science movement** is a driving force for Jupyter's popularity.<br>
> In part a response to the **reproducibility crisis in science** and the **statistical crisis in science** (aka data dredging, p-hacking) in science.<br>
> With Jupyter Notebooks contain **excutable code, equations, visualizations and narrative text**.<br>
> It is a **web application** (can run locally) with a simple and easy to use web interface.
> <img src="./images/narrative_new.svg" style="width: 300px;padding: 0; margin: 0;"><br>
> Jupyter supports a large number of programming languages (50+ e.g. Python, R, JavaScript)

The project is sponsered by large companies such as Google and Microsoft, and funders such as Alfred P. Sloan foundation. See link [jupyter.org/about](http://jupyter.org/about) for all sponsors.

#### Brief Instructions on How to Use Notebooks
- **Menu Help -> User Interface Tour** gives an overview of the user interface.
- **Code cells** contains the script code and have **In [x]** in the left margin.
  - **In []** indicates that the code cell hasn't been executed yet.
  - **In [n]** indicates that the code has been executed(n is an integer).
  - **In [\*]** indicates that the code is executing, or waiting to be executed (i.e. other cells are executing).
- **The current code** is highlighted with a blue border - you make it current by clicking on it.
- **SHIFT+ENTER** or **Play button** executes the current cell. Code cells aren't executed automatically.
- **Out[n]** indicates the output (or result) of a cell's execution and is directly below the executed cell.
- **SHIFT+ENTER** automatically selects the next code cell.
- **SHIFT+ENTER** can hence be used repeatedly to executes the code cells in sequence.
- **Menu Cell -> Run All** executes the entire notebook in a single step (can take some time to finish, notice how "In [\*]" indicators change to "In [n]" ).
- **Double-Click** on a cell to edit its content.
- **ESC key** Leaves edit mode (or just click on any other cell).
- **Kernel -> Restart** restarts server side kernel (use if notebook seems stuck)


### Risks
- The risk of using tools and methods **without fully understanding** them
- The risk of using tools and methods **for non-intended purposes or in new contexts**
- How to verify **performance** (correctness of result)
- Risk of **data dredging**, p-hacking, "the statistical crisis".
- The risk that **engineer makes micro-decisions** the researcher don'r know about
- The risk of **reading to much into visualizations** (networks, layouts, clusters).

### Challenges
- **What’s easy for humans can be extremely hard for computers**
- **Human-in-the-loop or supervised learning can be very expensive**
- Ambiguity and fuzziness of terms and phrases
- Poor data quality, errors in data, wrong data, missing data, ambigeous data
- Context, metadata, domain-specific data
- Data size (to much, to little)
- Computational methods requires a structured internal representation
- Internal models are a simplified views of the data
- etc...

### A sample high-level workflow

<img src="./images/text_analysis_workflow.svg" alt="" width="1200"/>

In [1]:
%%html
<style>
.jp-RenderedHTMLCommon td, .jp-RenderedHTMLCommon th, .jp-RenderedHTMLCommon tr {
    text-align: left;
    vertical-align: top;
}
</style>

# Sample text corpus processing

### TRY IT: Clean up the SOU text

In [2]:
# folded code
# -*- coding: utf-8 -*-
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import re
import string

punct_table = str.maketrans('', '', string.punctuation)

%matplotlib inline

hyphen_regexp = re.compile(r'\b(\w+)-\s*\r?\n\s*(\w+)\b', re.UNICODE)

def read_text_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        document = f.read()
    return document

def tokenize_and_sanitize(document, de_hyphen=True, min_length=3, only_isalpha=True, remove_puncts=True, to_lower=True, remove_stop=True):
        
    # hantera avstavningar
    if de_hyphen:
        document = re.sub(hyphen_regexp, r"\1\2\n", document)

    tokens = nltk.word_tokenize(document)
    
    # Ta bort ord kortare än tre tecken
    if min_length > 0:
        tokens = [ x for x in tokens if len([w for w in x if w.isalpha()]) > min_length ]
        
    # Ta bort ord som inte innehåller någon siffra eller bokstav
    if only_isalpha:
        tokens = [ x for x in tokens if x.isalpha() ]

    if remove_puncts:
        tokens = [ x.translate(punct_table) for x in tokens ]

    # Transformera till små bokstäver
    if to_lower:
        tokens = [ x.lower() for x in tokens ]
        
    # Ta bort de vanligaste stoporden
    if remove_stop:
        stopwords = nltk.corpus.stopwords.words('swedish')
        tokens = [ x for x in tokens if x not in stopwords ]
        
    return [ x for x in tokens if len(x) > 0 ]

def plot_xy_data(data, title='', xlabel='', ylabel='', **kwargs):
    
    x = list(data[0])
    y = list(data[1])
    labels = x

    plt.figure(figsize=(8, 8 / 1.618))
    plt.plot(x, y, 'ro', **kwargs)
    plt.xticks(x, labels, rotation='45')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plt.show()
    
container=dict(
    display_type=widgets.Dropdown(
        description='Show',
        value='statistics',
        options={
            'Source text': 'source_text',
            'Sanitized text': 'sanitized_text',
            'Statistics': 'statistics'           
    }),
    min_length=widgets.IntSlider(value=0, min=0, max=5, step=1, description='Min alpha', tooltip='Min number of alphabetic characters'),
    de_hyphen=widgets.ToggleButton(value=False, description='Dehyphen', disabled=False, tooltip='Fix hyphens', icon=''),
    to_lower=widgets.ToggleButton(value=False, description='Lowercase', disabled=False, tooltip='Transform text to lowercase', icon=''),
    remove_stop=widgets.ToggleButton(value=False, description='No stopwords', disabled=False, tooltip='Remove stopwords', icon=''),
    only_isalpha=widgets.ToggleButton(value=False, description='Only alpha', disabled=False, tooltip='Keep only alphabetic words', icon=''),
    remove_puncts=widgets.ToggleButton(value=False, description='Remove puncts.', disabled=False, tooltip='Remove punctioations characters', icon=''),
    progress=widgets.IntProgress(value=0, min=0, max=5, step=1, description='' )
)

output1 = widgets.Output() #layout={'border': '1px solid black'})
output2 = widgets.Output() #layout={'border': '1px solid black'})
default_output = None

tokens = []

def display_document(display_type, to_lower, remove_stop, only_isalpha, remove_puncts, min_length, de_hyphen, word_count=500):

    global tokens
    
    p =  container['progress']
    p.value = 0
    try:
        output1.clear_output()
        output2.clear_output()
        default_output.clear_output()
        document = read_text_file('./data/urn-nbn-se-kb-digark-2106487.txt')
        p.value = p.value + 1

        if display_type == 'source_text':
            # Utskrift av de första och sista 250 tecknen:
            with output1:
                print('{}\n.................\n(NOT SHOWN TEXT)\n.................\n{}'.format(document[:2500], document[-250:]))
            p.value = p.value + 1
            return
        
        p.value = p.value + 1

        tokens = tokenize_and_sanitize(
            document,
            de_hyphen=de_hyphen,
            min_length=min_length,
            only_isalpha=only_isalpha,
            remove_puncts=remove_puncts,
            to_lower=to_lower,
            remove_stop=remove_stop
        )

        if display_type in ['sanitized_text', 'statistics']:
            
            p.value = p.value + 1
            
            if display_type == 'sanitized_text':
                with output1:
                    display('{}\n.................\n(NOT SHOWN TEXT)\n.................\n{}'.format(
                        ' '.join(tokens[:word_count]),
                        ' '.join(tokens[-word_count:])
                    ))
                p.value = p.value + 1
                return
            
            if display_type == 'statistics':

                wf = nltk.FreqDist(tokens)
                p.value = p.value + 1
                
                with output1:
                    
                    df = pd.DataFrame(wf.most_common(25), columns=['token','count'])
                    display(df)
                
                with output2:
                    
                    print('Antal ord (termer): {}'.format(wf.N()))
                    print('Antal unika termer (vokabulär): {}'.format(wf.B()))
                    print(' ')
                    
                    data = list(zip(*wf.most_common(25)))
                    plot_xy_data(data, title='Word distribution', xlabel='Word', ylabel='Word count')
                    
                    wf = nltk.FreqDist([len(x) for x in tokens])
                    data = list(zip(*wf.most_common(25)))
                    plot_xy_data(data, title='Word length distribution', xlabel='Word length', ylabel='Word count')
    
    except Exception as ex:
        raise
        
    finally:
        p.value = 0

i_widgets = widgets.interactive(display_document, **container)
default_output = i_widgets.children[-1]
display(widgets.VBox([
    widgets.HBox([
        container['display_type'],
        container['to_lower'],
        container['remove_stop'],
        container['de_hyphen'],
        container['only_isalpha'],
        container['remove_puncts']
    ]),
    widgets.HBox([container['min_length'], container['progress']]),
    widgets.HBox([output1, output2]),
    default_output
]))

i_widgets.update()


FileNotFoundError: [Errno 2] No such file or directory: './data/urn-nbn-se-kb-digark-2106487.txt'

### TRY IT: Språkbanken NER tagging av SOU

In [4]:
import nltk
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline

import pandas as pd

entities = pd.read_csv('./data/SOU_1990_total_ner_extracted.csv', sep='\t',
                       names=['filename', 'year', 'location', 'categories', 'entity'])

entities['document_id'] = entities.filename.apply(lambda x: int(x.split('_')[1]))
entities['categories'] = entities.categories.str.replace('/', ' ')
entities['category'] = entities.categories.str.split(' ').str.get(0)
entities['sub_category'] = entities.categories.str.split(' ').str.get(1)

entities.drop(['location', 'categories'], inplace=True, axis=1)

document_names = pd.read_csv('./data/SOU_1990_index.csv',
                             sep='\t',
                             names=['year', 'sequence_id', 'report_name']).set_index('sequence_id')

def plot_freqdist(wf, n=25, **kwargs):
    data = list(zip(*wf.most_common(n)))
    x = list(data[0])
    y = list(data[1])
    labels = x

    plt.figure(figsize=(13, 13/1.618))
    plt.plot(x, y, '--ro', **kwargs)
    plt.xticks(x, labels, rotation='45')
    plt.show()

doc_names = { v: k for k, v in document_names.report_name.to_dict().items()}
doc_names['** ALL DOCUMENTS **'] = 0
@widgets.interact(category=entities.category.unique())
def display_most_frequent_pos_tags(document_id=doc_names, category='LOC', top=10):
    global entities
    locations = entities
    if document_id > 0:
        locations = locations.loc[locations.document_id==document_id]
    locations = locations.loc[locations.category==category]['entity']
    location_freqs = nltk.FreqDist(locations)
    #location_freqs.tabulate()
    plot_freqdist(location_freqs, n=top)

# display_most_frequent_pos_tags()

FileNotFoundError: File b'./data/SOU_1990_total_ner_extracted.csv' does not exist

### <span style='color:blue'>MANDATORY STEP</span> Setup and Initialize the Notebook
Use the **play** button, or press **Shift-Enter** to execute a code cell (select it first). The code imports Python libraries and frameworks, and initializes the notebook.

In [ ]:
# Folded Code
%load_ext autoreload
%autoreload 2

import common.utility
from common.model_utility import ModelUtility
from common.plot_utility import layout_algorithms, PlotNetworkUtility
import common.widgets_utility as wf
from common.network_utility import NetworkUtility, DISTANCE_METRICS, NetworkMetricHelper
#import common.vectorspace_utility

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import os
import glob
import math
import types
import ipywidgets as widgets
import logging
import bokeh.models as bm
import bokeh.palettes
import pandas as pd
import numpy as np

from pivottablejs import pivot_ui
from IPython.display import display, HTML, clear_output, IFrame
from itertools import product
from bokeh.io import output_file, push_notebook
from bokeh.core.properties import value, expr
from bokeh.transform import transform, jitter
from bokeh.layouts import row, column, widgetbox
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.models import ColumnDataSource, CustomJS

logger = logging.getLogger('explore-topic-models')
TOOLS = "pan,wheel_zoom,box_zoom,reset,previewsave"
AGGREGATES = { 'mean': np.mean, 'sum': np.sum, 'max': np.max, 'std': np.std }

output_notebook()

pd.set_option('precision', 10)

### <span style='color:blue'>MANDATORY STEP</span> Select LDA Topic Model
- Select one of the previously computed and prepared topic models that you wan't to use in subsequent steps.
- Models are computed in batch in accordance to 
<a href="./images/workflow-prepare.svg">process flow</a> used in the *Digitala modeller* project.
- Note that subsequent code cells are NOT updated (executed) automatically when a new model is selected.
- Use the **play** button, or press **Shift-Enter** to execute the selected cell.

In [ ]:
# Hidden code: Select current model state
class ModelState:
    
    def __init__(self, data_folder):
        
        self.data_folder = data_folder
        self.basenames = ModelUtility.get_model_names(data_folder)
        self.basename = self.basenames[0]
        self.on_set_model_callback = None
        
    def set_model(self, basename=None):

        basename = basename or self.basename
        
        self.basename = basename
        self.topic_keys = ModelUtility.get_topic_keys(self.data_folder, basename)
        state.max_alpha = self.topic_keys.alpha.max()
        self.topic_overview = ModelUtility\
            .get_result_model_sheet(self.data_folder, basename, 'topic_tokens')
        self.document_topic_weights = ModelUtility\
            .get_result_model_sheet(self.data_folder, basename, 'doc_topic_weights')\
            .drop('Unnamed: 0', axis=1, errors='ignore')
        self.topic_token_weights = ModelUtility\
            .get_result_model_sheet(self.data_folder, basename, 'topic_token_weights')\
            .drop('Unnamed: 0', axis=1, errors='ignore')\
            .dropna(subset=['token'])
        self._years = list(range(
            self.document_topic_weights.year.min(), self.document_topic_weights.year.max() + 1))
        self.min_year = min(self._years)
        self.max_year = max(self._years)
        self.years = [None] + self._years
        self.n_topics = self.topic_overview.topic_id.max() + 1
        # https://stackoverflow.com/questions/44561609/how-does-mallet-set-its-default-hyperparameters-for-lda-i-e-alpha-and-beta
        self.initial_alpha = 0.0  # 5.0 / self.n_topics if 'mallet' in state.basename else 1.0 / self.n_topics
        self.initial_beta = 0.0  # 0.01 if 'mallet' in basename else 1.0 / self.n_topics
        self._lda = None
        self._topic_titles = None
        self.corpus_documents = ModelUtility.get_corpus_documents(self.data_folder, self.basename).set_index('document_id')
        print("Current model: " + self.basename.upper())
        
        if self.on_set_model_callback is not None:
            self.on_set_model_callback(self)
            
        # _fix_topictokens()
        return self
    
    #def get_document_topic_weights(self, year=None, topic_id=None):
    #    df = self.document_topic_weights
    #    if year is None and topic_id is None:
    #        return df
    #    if topic_id is None:
    #        return df[(df.year == year)]
    #    if year is None:
    #        return df[(df.topic_id == topic_id)]
    #    return df[(df.year == year)&(df.topic_id == topic_id)]
    
    def get_unique_topic_ids(self):
        return self.document_topic_weights['topic_id'].unique()
    
    #def get_topic_weight_by_year_or_document(self, key='mean', pivot_column=None):
    #    
    #    if pivot_column is None:
    #        pivot_column = 'year' if year is None else 'document_id'    
    #        
    #    df = self.document_topic_weights(year) \
    #        .groupby([pivot_column,'topic_id']) \
    #        .agg(AGGREGATES[key])[['weight']].reset_index()
    #    return df, pivot_column
    
    #return self.get_document_topic_weight_by_pivot_column(pivot_column, key, filter={'column': 'year', 'values': [year]})
    
    def get_document_topic_weight_by_filter(self, filters=None):
        df = self.document_topic_weights.query('weight > 0')
        for filter in (filters or []):
            if 'query' in filter.keys():
                df = df.query(filter['query'])
            elif isinstance(filter['value'], str):
                df = df[(df[filter['column']]==filter['value'])]
            elif isinstance(filter['value'], list):
                df = df[(df[filter['column']].isin(filter['value']))]
        return df
    
    def get_document_topic_weight_by_pivot_column(self, pivot_column, key='mean', filters=None):
        df = self.get_document_topic_weight_by_filter(filters)
        df = df.groupby([pivot_column, 'topic_id'])\
               .agg(AGGREGATES[key])[['weight']].reset_index()
        return df[df.weight > 0]
    
    def get_topic_tokens_dict(self, topic_id, n_top=200):
        return self.get_topic_tokens(topic_id)\
            .sort_values(['weight'], ascending=False)\
            .head(n_top)[['token', 'weight']]\
            .set_index('token').to_dict()['weight']

    def compute_topic_terms_vector_space(self, n_words=100):
        '''
        Create an align topic-term vector space of top n_words from each topic
        '''
        unaligned_vector_dicts = ( self.get_topic_tokens_dict(topic_id, n_words) for topic_id in range(0, self.n_topics) )
        X, feature_names = ModelUtility.compute_and_align_vector_space(unaligned_vector_dicts)
        return X, feature_names

    def get_lda(self):
        raise Exception("Use of LDA model disabled in this Notebook")
        '''
        Get gensim model. Only used for pyLDAvis display
        '''
        if self._lda is None:
            filename = os.path.join(self.data_folder, self.basename, 'gensim_model_{}.gensim.gz'.format(self.basename))
            if os.path.isfile(filename):
                self._lda = LdaModel.load(filename)
                print('LDA model loaded...')
            else:
                print('LDA not found on disk...')
        return self._lda 
    
    def get_topic_titles(self, n_words=100, cache=True):
        if cache and self._topic_titles is not None:
            return self._topic_titles
        _topic_titles = ModelUtility.get_topic_titles(state.topic_token_weights, n_words=n_words)
        self._topic_titles = _topic_titles if cache else None
        return _topic_titles
    
    def get_topic_tokens(self, topic_id, max_n_words=500):
        tokens = state.topic_token_weights\
            .loc[lambda x: x.topic_id == topic_id]\
            .sort_values('weight',ascending=False)[:max_n_words]
        return tokens
    
    def get_topic_alphas(self):
        tokens = state.topic_token_weights\
            .loc[lambda x: x.topic_id == topic_id]\
            .sort_values('weight',ascending=False)[:max_n_words]
        alpas = ModelUtility.get_topic_alphas
        return tokens
    
    def get_topic_year_aggregate_weights(self, fn, threshold):
        df = self.document_topic_weights[(self.document_topic_weights.weight > 0.001)]
        df = df.groupby(['year', 'topic_id']).agg(fn)['weight'].reset_index()
        df = df[(df.weight>=threshold)]
        return df
    
    def get_topic_proportions(self):
        corpus_documents = self.get_corpus_documents()
        document_topic_weights = self.document_topic_weights
        topic_proportion = ModelUtility.compute_topic_proportions(document_topic_weights, corpus_documents)
        return topic_proportion
    
    def get_corpus_documents(self):
        #if self.corpus_documents is None:
        #    self.corpus_documents = ModelUtility.get_corpus_documents(self.data_folder, self.basename)
        return self.corpus_documents

    def on_set_model(self, callback):
        self.on_set_model_callback = callback
        return self
        
def on_set_model_handler(state):

    if 'report_name' in state.corpus_documents:
        return
    
    state.source_documents = pd.read_csv('data/SOU_1990_index.csv', sep='\t', header=None, names=['year', 'report_id', 'report_name'])
    state.corpus_documents['report_id'] = state.corpus_documents.document.str.split('_').apply(lambda x: x[1]).astype(np.int64)
    state.corpus_documents['report_name'] = pd.merge(state.corpus_documents, state.source_documents, how='inner', on=['year', 'report_id']).report_name
    state.corpus_documents['report_name'] = state.corpus_documents.apply(lambda x: '{}-{} {}'.format(x['year'], x['report_id'], x['report_name'])[:50], axis=1)
    state.document_topic_weights['report_name'] = pd.merge(state.document_topic_weights, state.corpus_documents, left_on='document_id', right_index=True).report_name

def select_model_main(state):
    
    basename_widget = widgets.Dropdown(
        options=state.basenames,
        value=state.basename,
        description='Topic model',
        disabled=False,
        layout=widgets.Layout(width='75%')
    )
    
    w = widgets.interactive(state.set_model, basename=basename_widget, state=widgets.fixed(state))
    display(widgets.VBox((basename_widget,) + (w.children[-1],)))
    w.update()

state = ModelState('./data').on_set_model(on_set_model_handler)

select_model_main(state)


### Topic-Word Distribution - Wordcloud and Table

In [31]:
# Display LDA topic's token wordcloud
opts = { 'max_font_size': 100, 'background_color': 'white', 'width': 900, 'height': 600 }

import wordcloud
import matplotlib.pyplot as plt

def display_topic_distribution_widgets(callback, state, text_id, output_options=None, word_count=(1, 100, 50)):
    
    output_options = output_options or []
    wc = wf.BaseWidgetUtility(
        n_topics=state.n_topics,
        text_id=text_id,
        text=wf.create_text_widget(text_id),
        topic_id=widgets.IntSlider(
            description='Topic ID', min=0, max=state.n_topics - 1, step=1, value=0, continuous_update=False),
        word_count=widgets.IntSlider(
            description='#Words', min=word_count[0], max=word_count[1], step=1, value=word_count[2], continuous_update=False),
        output_format=wf.create_select_widget('Format', output_options, default=output_options[0], layout=widgets.Layout(width="200px")),
        progress = widgets.IntProgress(min=0, max=4, step=1, value=0, layout=widgets.Layout(width="95%"))
    )

    wc.prev_topic_id = wc.create_prev_id_button('topic_id', state.n_topics)
    wc.next_topic_id = wc.create_next_id_button('topic_id', state.n_topics)

    iw = widgets.interactive(
        callback,
        topic_id=wc.topic_id,
        n_words=wc.word_count,
        output_format=wc.output_format,
        widget_container=widgets.fixed(wc)
    )

    display(widgets.VBox([
        wc.text,
        widgets.HBox([wc.prev_topic_id, wc.next_topic_id, wc.topic_id, wc.word_count, wc.output_format]),
        wc.progress,
        iw.children[-1]
    ]))

    iw.update()

def plot_wordcloud(df_data, token='token', weight='weight', figsize=(14, 14/1.618), **args):
    token_weights = dict({ tuple(x) for x in df_data[[token, weight]].values })
    image = wordcloud.WordCloud(**args,)
    image.fit_words(token_weights)
    plt.figure(figsize=figsize) #, dpi=100)
    plt.imshow(image, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    
def display_wordcloud(topic_id=0, n_words=100, output_format='Wordcloud', widget_container=None):
    widget_container.progress.value = 1
    df_temp = state.topic_token_weights.loc[(state.topic_token_weights.topic_id == topic_id)]
    tokens = state.get_topic_titles(n_words=n_words, cache=True).iloc[topic_id]
    widget_container.value = 2
    widget_container.text.value = 'ID {}: {}'.format(topic_id, tokens)
    if output_format == 'Wordcloud':
        plot_wordcloud(df_temp, 'token', 'weight', max_words=n_words, **opts)
    elif output_format == 'Table':
        widget_container.progress.value = 3
        df_temp = state.get_topic_tokens(topic_id, n_words)
        widget_container.progress.value = 4
        display(HTML(df_temp.to_html()))
    else:
        display(pivot_ui(state.get_topic_tokens(topic_id, n_words)))
    widget_container.progress.value = 0

display_topic_distribution_widgets(display_wordcloud, state, 'tx02', ['Wordcloud', 'Table', 'Pivot'])


### Topic-Word Distribution - Chart
The following chart shows the word distribution for each selected topic. You can zoom in on the left chart. The distribution seems to follow [Zipf's law](https://en.wikipedia.org/wiki/Zipf%27s_law) as (perhaps) expected.

In [32]:
# Display topic's word distribution

def plot_topic_word_distribution(tokens, **args):

    source = ColumnDataSource(tokens)

    p = figure(toolbar_location="right", **args)

    cr = p.circle(x='xs', y='ys', source=source)

    label_style = dict(level='overlay', text_font_size='8pt', angle=np.pi/6.0)

    text_aligns = ['left', 'right']
    for i in [0, 1]:
        label_source = ColumnDataSource(tokens.iloc[i::2])
        labels = bm.LabelSet(x='xs', y='ys', text_align=text_aligns[i], text='token', text_baseline='middle',
                          y_offset=5*(1 if i == 0 else -1),
                          x_offset=5*(1 if i == 0 else -1),
                          source=label_source, **label_style)
        p.add_layout(labels)

    p.xaxis[0].axis_label = 'Token #'
    p.yaxis[0].axis_label = 'Probability%'
    p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "6pt"
    p.axis.major_label_standoff = 0
    return p

def plot_topic_tokens_charts(tokens, flag=True):

    if flag:
        left = plot_topic_word_distribution(tokens, plot_width=1000, plot_height=500, title='', tools='box_zoom,wheel_zoom,pan,reset')
        show(left)
        return

    left = plot_topic_word_distribution(tokens, plot_width=450, plot_height=500, title='', tools='box_zoom,wheel_zoom,pan,reset')
    right = plot_topic_word_distribution(tokens, plot_width=450, plot_height=500, title='', tools='pan')

    source = ColumnDataSource({'x':[], 'y':[], 'width':[], 'height':[]})
    left.x_range.callback = create_js_callback('x', 'width', source)
    left.y_range.callback = create_js_callback('y', 'height', source)

    rect = bm.Rect(x='x', y='y', width='width', height='height', fill_alpha=0.0, line_color='blue', line_alpha=0.4)
    right.add_glyph(source, rect)

    show(row(left, right))

def display_topic_tokens(topic_id=0, n_words=100, output_format='Chart', widget_container=None):
    widget_container.forward()
    tokens = state.get_topic_tokens(topic_id=topic_id).\
        copy()\
        .drop('topic_id', axis=1)\
        .assign(weight=lambda x: 100.0 * x.weight)\
        .sort_values('weight', axis=0, ascending=False)\
        .reset_index()\
        .head(n_words)
    if output_format == 'Chart':
        widget_container.forward()
        tokens = tokens.assign(xs=tokens.index, ys=tokens.weight)
        plot_topic_tokens_charts(tokens)
        widget_container.forward()
    elif output_format == 'Table':
        #display(tokens)
        display(HTML(tokens.to_html()))
    else:
        display(pivot_ui(tokens))
    widget_container.reset()
        
display_topic_distribution_widgets(display_topic_tokens, state, 'wc01', ['Chart', 'Table'])


### Topic's Trend Over Time or Documents
- Displays topic's share over documents or time.
- Note that source documents (i.e. SOU reports) are splitted into 1000 word chunks (LDA document) by the topic modelling process
- If "SOU Report" or "Year" is selected then the **max** or **mean** weight is selected from corresponding LDA documents

In [33]:
# Plot a topic's yearly weight over time in selected LDA topic model
import numpy as np
import math
import bokeh.plotting
from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid
from bokeh.models.glyphs import VBar
from bokeh.io import curdoc, show

def plot_topic_trend(df, pivot_column, value_column, x_label=None, y_label=None):

    xs = df[pivot_column].astype(np.str)
    p = bokeh.plotting.figure(x_range=xs, plot_width=1000, plot_height=700, title='', tools=TOOLS, toolbar_location="right")

    glyph = p.vbar(x=xs, top=df[value_column], width=0.5, fill_color="#b3de69")
    p.xaxis.major_label_orientation = math.pi/4
    p.xgrid.grid_line_color = None
    p.xaxis[0].axis_label = (x_label or '').title()
    p.yaxis[0].axis_label = (y_label or '').title()
    p.y_range.start = 0.0
    #p.y_range.end = 1.0
    p.x_range.range_padding = 0.01
    return p

def display_topic_trend(topic_id, pivot_config, value_column, widgets_container, output_format='Chart', state=None, threshold=0.01):
    
    pivot_column = pivot_config['pivot_column']
    tokens = state.get_topic_titles(n_words=200, cache=True).iloc[topic_id]
    widgets_container.text.value = 'ID {}: {}'.format(topic_id, tokens)
    value_column = value_column if pivot_column is not None else 'weight'
    
    df = state.document_topic_weights[(state.document_topic_weights.topic_id==topic_id)]
    
    if pivot_column is not None:
        df = df.groupby([pivot_column]).agg([np.mean, np.max])['weight'].reset_index()
        df.columns = [pivot_column, 'mean', 'max' ]
        df = df[(df[value_column] > threshold)]
        
    if output_format == 'Table':
        display(df)
    else:
        x_label = pivot_column.title()
        y_label = value_column.title() + ('weight' if value_column != 'weight' else '')
        p = plot_topic_trend(df, pivot_column, value_column, x_label=x_label, y_label=y_label)
        show(p)

def create_topic_trend_widgets(state):
    pivot_options = {
        '': { 'pivot_column': None, 'filter': None },
        'SOU Report': { 'pivot_column': 'report_name', 'filter': None },
        'Year': { 'pivot_column': 'year', 'filter': None },
        'LDA Document': { 'pivot_column': 'document_id', 'filter': None }
    } 
    wc = wf.BaseWidgetUtility(
        n_topics=state.n_topics,
        text_id='topic_share_plot',
        text=wf.create_text_widget('topic_share_plot'),
        #year=wf.create_select_widget('Year', options=state.years, value=state.years[-1]),
        pivot_config=widgets.Dropdown(
            options=pivot_options,
            value=pivot_options['SOU Report'],
            description='Group by'
        ),
        threshold=widgets.FloatSlider(description='Threshold', min=0.0, max=0.25, step=0.01, value=0.10, continuous_update=False),
        topic_id=widgets.IntSlider(description='Topic ID', min=0, max=state.n_topics - 1, step=1, value=0, continuous_update=False),
        output_format=wf.create_select_widget('Format', ['Chart', 'Table'], default='Chart'),
        progress=widgets.IntProgress(min=0, max=4, step=1, value=0, layout=widgets.Layout(width="50%")),
        aggregate=widgets.Dropdown(options=['max', 'mean'], value='max', description='Aggregate')
    )

    wc.prev_topic_id = wc.create_prev_id_button('topic_id', state.n_topics)
    wc.next_topic_id = wc.create_next_id_button('topic_id', state.n_topics)

    iw = widgets.interactive(
        display_topic_trend,
        topic_id=wc.topic_id,
        pivot_config=wc.pivot_config,
        value_column=wc.aggregate,
        widgets_container=widgets.fixed(wc),
        output_format=wc.output_format,
        state=widgets.fixed(state),
        threshold=wc.threshold
    )
    display(widgets.VBox([
        wc.text,
        widgets.HBox([wc.prev_topic_id, wc.next_topic_id, wc.pivot_config, wc.aggregate, wc.output_format]),
        widgets.HBox([wc.topic_id, wc.threshold, wc.progress]),
        iw.children[-1]
    ]))
    
    iw.update()
    
create_topic_trend_widgets(state)

### Topic to Document Network
The green nodes are documents, and blue nodes are topics. The edges (lines) indicates the strength of a topic in the connected document. The width of the edge is proportinal to the strength of the connection. Note that only edges with a strength above the certain threshold are displayed.

In [34]:
# Visualize year-to-topic network by means of topic-document-weights
     
def plot_topic_year_network(network, layout, scale=1.0, titles=None):

    year_nodes, topic_nodes = NetworkUtility.get_bipartite_node_set(network, bipartite=0)  
    
    year_source = NetworkUtility.get_node_subset_source(network, layout, year_nodes)
    topic_source = NetworkUtility.get_node_subset_source(network, layout, topic_nodes)
    lines_source = NetworkUtility.get_edges_source(network, layout, scale=6.0, normalize=False)
    
    edges_alphas = NetworkMetricHelper.compute_alpha_vector(lines_source.data['weights'])
    
    lines_source.add(edges_alphas, 'alphas')
    
    p = figure(plot_width=1000, plot_height=600, x_axis_type=None, y_axis_type=None, tools=TOOLS)
    
    r_lines = p.multi_line(
        'xs', 'ys', line_width='weights', alpha='alphas', color='black', source=lines_source
    )
    r_years = p.circle(
        'x','y', size=40, source=year_source, color='lightgreen', level='overlay', line_width=1,alpha=1.0
    )
    
    r_topics = p.circle('x','y', size=25, source=topic_source, color='skyblue', level='overlay', alpha=1.00)
    
    p.add_tools(bm.HoverTool(renderers=[r_topics], tooltips=None, callback=wf.WidgetUtility.\
        glyph_hover_callback(topic_source, 'node_id', text_ids=titles.index, text=titles, element_id='nx_id1'))
    )

    text_opts = dict(
        x='x', y='y', text='name', level='overlay',
        x_offset=0, y_offset=0, text_font_size='8pt'
    )
    
    p.add_layout(
        bm.LabelSet(
            source=year_source, text_color='black', text_align='center', text_baseline='middle', **text_opts
        )
    )
    p.add_layout(
        bm.LabelSet(
            source=topic_source, text_color='black', text_align='center', text_baseline='middle', **text_opts
        )
    )
    
    return p

def main_topic_year_network(state):
    
    wc = wf.BaseWidgetUtility(
        n_topics=state.n_topics,
        text_id='nx_id1',
        text=wf.create_text_widget('nx_id1', style="display: inline; height='400px'"),
        year=widgets.IntSlider(description='Year', min=state.min_year, max=state.max_year, step=1, value=state.min_year, continues_update=False),
        pivot_column=widgets.Dropdown(
            options={
                'SOU report': 'report_name',
                'Year': 'year'
            },
            value='report_name',
            description='Pivot'
        ),
        scale=widgets.FloatSlider(description='Scale', min=0.0, max=1.0, step=0.01, value=0.1, continues_update=False),
        threshold=widgets.FloatSlider(description='Threshold', min=0.0, max=1.0, step=0.01, value=0.50, continues_update=False),
        output_format=widgets.Dropdown(
            options={'Network': 'network', 'Table': 'table'},
            value='network',
            description='Output'
        ),
        layout=widgets.Dropdown(
            options=list(layout_algorithms.keys()),
            value='Fruchterman-Reingold',
            description='Layout'
        ),
        progress=widgets.IntProgress(min=0, max=4, step=1, value=0, layout=widgets.Layout(width="40%"))
    ) 
    
    wc.previous = wc.create_prev_id_button('year', 10000)
    wc.next = wc.create_next_id_button('year', 10000)    
    
    def display_topic_year_network(
        layout_algorithm,
        threshold=0.50,
        scale=1.0,
        pivot_column='report_name',
        year=None,
        output_format='network'
    ):
        wc.progress.value = 1
        
        titles = state.get_topic_titles()
        filters = []
        if year is not None:
            filters = [ { 'column': 'year', 'value': year }]
        filters = filters + [ { 'query': 'weight >= {}'.format(threshold) } ]
        df = state.get_document_topic_weight_by_pivot_column(pivot_column, key='max', filters=filters)
        df = df[df.weight > threshold]
        
        wc.progress.value = 2

        network = NetworkUtility.create_bipartite_network(df, pivot_column, 'topic_id')
        
        wc.progress.value = 3

        if output_format == 'network':
            
            args = PlotNetworkUtility.layout_args(layout_algorithm, network, scale)
            layout = (layout_algorithms[layout_algorithm])(network, **args)
            
            wc.progress.value = 4
            
            p = plot_topic_year_network(network, layout, scale=scale, titles=titles)
            show(p)

        elif output_format == 'table':
            print(df.shape)
            display(df)
        else:
            display(pivot_ui(df))

        wc.progress.value = 0

    iw = widgets.interactive(
        display_topic_year_network,
        layout_algorithm=wc.layout,
        threshold=wc.threshold,
        scale=wc.scale,
        pivot_column=wc.pivot_column,
        year=wc.year,
        output_format=wc.output_format
    )

    display(widgets.VBox([
        wc.text,
        widgets.HBox([wc.layout, wc.year, wc.previous, wc.next]),
        widgets.HBox([wc.pivot_column, wc.scale]),
        widgets.HBox([wc.output_format, wc.threshold, wc.progress]),
        iw.children[-1]
    ]))
    iw.update()
    
main_topic_year_network(state)


### Topic Trends - Heatmap
- The topic shares  displayed as a scattered heatmap plot using gradient color based on topic's weight in document.
- [Stanford’s Termite software](http://vis.stanford.edu/papers/termite) uses a similar visualization.

In [35]:
# plot_topic_relevance_by_year

def setup_glyph_coloring(df):
    max_weight = df.weight.max()
    #colors = list(reversed(bokeh.palettes.Greens[9]))
    colors = ["#efefef", "#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878",
              "#933b41", "#550b1d"]
    mapper = bm.LinearColorMapper(palette=colors, low=df.weight.min(), high=max_weight)
    color_transform = transform('weight', mapper)
    color_bar = bm.ColorBar(color_mapper=mapper, location=(0, 0),
                         ticker=bm.BasicTicker(desired_num_ticks=len(colors)),
                         formatter=bm.PrintfTickFormatter(format=" %5.2f"))
    return color_transform, color_bar

def plot_topic_relevance_by_year(df, xs, ys, flip_axis, glyph, titles, text_id):

    line_height = 7
    if flip_axis is True:
        xs, ys = ys, xs
        line_height = 10
    
    ''' Setup axis categories '''
    x_range = list(map(str, df[xs].unique()))
    y_range = list(map(str, df[ys].unique()))
    
    ''' Setup coloring and color bar '''
    color_transform, color_bar = setup_glyph_coloring(df)
    
    source = ColumnDataSource(df)

    plot_height = max(len(y_range) * line_height, 500)
    
    p = figure(title="Topic heatmap", tools=TOOLS, toolbar_location="right", x_range=x_range,
           y_range=y_range, x_axis_location="above", plot_width=1000, plot_height=plot_height)

    args = dict(x=xs, y=ys, source=source, alpha=1.0, hover_color='red')
    
    if glyph == 'Circle':
        cr = p.circle(color=color_transform, **args)
    else:
        cr = p.rect(width=1, height=1, line_color=None, fill_color=color_transform, **args)

    p.x_range.range_padding = 0
    p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "5pt"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = 1.0
    p.add_layout(color_bar, 'right')
    
    p.add_tools(bm.HoverTool(tooltips=None, callback=wf.WidgetUtility.glyph_hover_callback(
        source, 'topic_id', titles.index, titles, text_id), renderers=[cr]))
    
    return p

def topic_heatmap_main(state):
    
    def display_topic_relevance_by_year(state, key='max', pivot_column=None, year=None, flip_axis=False, glyph='Circle', wdgs=None):
        
        try:
            wdgs.reset()
            wdgs.forward()
            
            titles = ModelUtility.get_topic_titles(state.topic_token_weights, n_words=100)
            wdgs.forward()

            year = (year or 0)
            
            pivot_column = 'year' if year > 0 else (pivot_column or 'report_name')
            filters = [{'column': 'year', 'values': [year]}] if year > 0 else []
            
            df = state.get_document_topic_weight_by_pivot_column(pivot_column, key, filters=filters)
            
            wdgs.forward()
            
            df[pivot_column] = df[pivot_column].astype(str)
            df['topic_id'] = df.topic_id.astype(str)
            
            wdgs.forward()
            
            p = plot_topic_relevance_by_year(df, xs=pivot_column, ys='topic_id', flip_axis=flip_axis, glyph=glyph, titles=titles, text_id='topic_relevance')
            
            show(p)
            wdgs.reset()
        except Exception as ex:
            raise
            logger.error(ex)
        finally:
            wdgs.reset()

    wc = wf.BaseWidgetUtility(
        text_id='topic_relevance',
        text=wf.create_text_widget('topic_relevance'),
        year=widgets.Dropdown(options=state.years, value=None, description='Year', layout=widgets.Layout(width="140px")),
        pivot_column=widgets.Dropdown(
            options={
                'SOU report': 'report_name',
                # 'LDA document': 'document_id',
                'Year': 'year'
            },
            value='report_name',
            description='Pivot',
            layout=widgets.Layout(width="200px")
        ),
        aggregate=widgets.Dropdown(options=['max', 'mean'], value='max', description='Aggregate', layout=widgets.Layout(width="180px")),
        progress=widgets.IntProgress(min=0, max=4, step=1, value=0, layout=widgets.Layout(width="35%")),
        glyph=widgets.Dropdown(options=['Circle', 'Square'], value='Square', description='Glyph', layout=widgets.Layout(width="180px")),
        flip_axis=widgets.ToggleButton(value=True, description='Flip XY', tooltip='Flip X and Y axis', icon='', layout=widgets.Layout(width="80px"))
    )

    iw = widgets.interactive(
        display_topic_relevance_by_year,
        state=widgets.fixed(state),
        key=wc.aggregate,
        pivot_column=wc.pivot_column,
        year=wc.year,
        glyph=wc.glyph,
        flip_axis=wc.flip_axis,
        wdgs=widgets.fixed(wc)
    )

    display(widgets.VBox([
        widgets.HBox([wc.pivot_column, wc.year, wc.aggregate, wc.flip_axis, wc.glyph, wc.progress ]),
        wc.text,
        iw.children[-1]
    ]))

    iw.update()
            
topic_heatmap_main(state)